<a href="https://colab.research.google.com/github/L4ncelot1024/Learn_Deep_Learning_Le_Wagon/blob/main/Day4/CNN_LiveCode_Pretrained_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Live Code: Pretrained Models

Let's see in more details the classical CNN architectures which are popular and how to apply them !

In [ ]:
%tensorflow_version 2.x

In [ ]:
%matplotlib inline
import os
import zipfile

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

## Data Loading

Here we download and process the data like in the exercises

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2021-04-26 06:10:33--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 142.250.107.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  82.6MB/s    in 0.8s    

2021-04-26 06:10:34 (82.6 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our test cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')

# Directory with our test dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')

train_cat_fnames = os.listdir(train_cats_dir)

train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total test cat images: 500
total test dog images: 500


## Data Augmentation

Since the dataset is very small, we add a data augmentation step

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255, for the train generator we set
# the validation split.
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, 
                                   width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

# global settings
target_size = (224, 224)
batch_size = 20

# Flow training images in batches of 20 using train_datagen generator
print('Train Dataset')
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=target_size,  # All images will be resized to 150x150
        batch_size=batch_size,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

print('Test Dataset')
# Flow validation images in batches of 20 using val_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary')

Train Dataset
Found 2000 images belonging to 2 classes.
Test Dataset
Found 1000 images belonging to 2 classes.


## Pre-trained Model

One thing that is commonly done in computer vision is to take a model trained on a very large dataset, run it on your own, smaller dataset, and extract the intermediate representations (features) that the model generates. These representations are frequently informative for your own computer vision task, even though the task may be quite different from the problem that the original model was trained on. This versatility and repurposability of convnets is one of the most interesting aspects of deep learning.

We will apply this method using 4 popular architecture for which Keras provides pre-trained models (but there are way more to test [here](https://keras.io/api/applications/)

For each model, the strategy is the same: 
1. First, you need to adapt your image to those used for training the loaded model if it accepts only this side. Here this requires to re-run the initialisation of the data generator from above;
2. Then, you need to identify which intermediate layer of the model to use for feature extraction. A common practice is to use the output of the antepenultieme layer (before the classifier layer). You will remove layers defined in the network after this one.
3. Freeze the layers you don't want to finetune (here we won't do any fine-tuning, but in the exercise you see how to do it);
4. You then need to add a final layer to solve your task and output your expected target, here it will be a simple binary classifier layer.
5. Finally, you can train your model !


### VGG-16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

# Step 1 and 2
base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
# Step 3
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Step 4

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5 
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
# Step 5
vgghist = model.fit(train_generator, validation_data=test_generator, 
                    steps_per_epoch=100, epochs=10)

Epoch 1/10
100/100 [==============================] - 69s 323ms/step - loss: 0.9805 - acc: 0.6100 - val_loss: 0.3319 - val_acc: 0.8680
Epoch 2/10
100/100 [==============================] - 32s 321ms/step - loss: 0.4791 - acc: 0.7647 - val_loss: 0.2592 - val_acc: 0.9020
Epoch 3/10
100/100 [==============================] - 32s 321ms/step - loss: 0.4490 - acc: 0.8062 - val_loss: 0.3353 - val_acc: 0.8480
Epoch 4/10
100/100 [==============================] - 32s 322ms/step - loss: 0.3934 - acc: 0.8213 - val_loss: 0.2176 - val_acc: 0.9170
Epoch 5/10
100/100 [==============================] - 32s 323ms/step - loss: 0.3921 - acc: 0.8317 - val_loss: 0.2678 - val_acc: 0.8880
Epoch 6/10
100/100 [==============================] - 32s 321ms/step - loss: 0.3696 - acc: 0.8372 - val_loss: 0.2473 - val_acc: 0.8970
Epoch 7/10
100/100 [==============================] - 32s 321ms/step - loss: 0.3379 - acc: 0.8558 - val_loss: 0.2150 - val_acc: 0.9140
Epoch 8/10
100/100 [==============================] - 3

### Inception

In [ ]:
# Step 1
target_size = (150, 150)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, 
                                                    class_mode = 'binary', target_size = target_size)
test_generator = test_datagen.flow_from_directory(test_dir, batch_size = 20, 
                                                        class_mode = 'binary', target_size = target_size)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# Step 2
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = list(target_size) + [3] , 
                         include_top = False, weights = 'imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
# Step 3
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Step 4
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])


In [ ]:
inc_history = model.fit_generator(train_generator, validation_data=test_generator,
                                  steps_per_epoch = 100, epochs = 10)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
100/100 [==============================] - 56s 198ms/step - loss: 4.1352 - acc: 0.7667 - val_loss: 0.1446 - val_acc: 0.9530
Epoch 2/10
100/100 [==============================] - 19s 187ms/step - loss: 0.3763 - acc: 0.8979 - val_loss: 0.4961 - val_acc: 0.8950
Epoch 3/10
100/100 [==============================] - 19s 188ms/step - loss: 0.3648 - acc: 0.8996 - val_loss: 0.1306 - val_acc: 0.9560
Epoch 4/10
100/100 [==============================] - 19s 188ms/step - loss: 0.4215 - acc: 0.8998 - val_loss: 0.1560 - val_acc: 0.9530
Epoch 5/10
100/100 [==============================] - 19s 186ms/step - loss: 0.3238 - acc: 0.9069 - val_loss: 0.3476 - val_acc: 0.9100
Epoch 6/10
100/100 [==============================] - 19s 185ms/step - loss: 0.3206 - acc: 0.9093 - val_loss: 0.2014 - val_acc: 0.9490
Epoch 7/10
100/100 [==============================] - 18s 185ms/step - loss: 0.2809 - acc: 0.9268 - val_loss: 0.1436 - val_acc: 0.9550
Epoch 8/10
100/100 [==============================] - 1

### ResNet50

In [ ]:
# Step 1
target_size = (224, 224)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=target_size,  # All images will be resized to 150x150
        batch_size=batch_size,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# Step 2
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=list(target_size) + [3] , include_top=False, weights="imagenet")

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
# Step 3
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Step 4 - NB: here we use the tensorflow Sequential framework to simplify the model creation

from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

base_model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
# Step 5
resnet_history = base_model.fit(train_generator, validation_data = test_generator, 
                                steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 49s 380ms/step - loss: 0.7490 - acc: 0.8498 - val_loss: 14.0817 - val_acc: 0.5000
Epoch 2/10
100/100 [==============================] - 37s 370ms/step - loss: 0.2975 - acc: 0.9391 - val_loss: 5.3193 - val_acc: 0.5000
Epoch 3/10
100/100 [==============================] - 37s 370ms/step - loss: 0.2224 - acc: 0.9540 - val_loss: 9.1252 - val_acc: 0.5000
Epoch 4/10
100/100 [==============================] - 37s 368ms/step - loss: 0.1860 - acc: 0.9587 - val_loss: 1.7131 - val_acc: 0.4880
Epoch 5/10
100/100 [==============================] - 37s 371ms/step - loss: 0.1596 - acc: 0.9638 - val_loss: 4.4278 - val_acc: 0.5020
Epoch 6/10
100/100 [==============================] - 37s 370ms/step - loss: 0.1419 - acc: 0.9704 - val_loss: 4.9223 - val_acc: 0.5010
Epoch 7/10
100/100 [==============================] - 37s 369ms/step - loss: 0.1179 - acc: 0.9766 - val_loss: 0.8491 - val_acc: 0.7830
Epoch 8/10
100/100 [==============================] - 

### EfficientNet

In [ ]:
# Step 0 - For this model we need to add an extra dependency
!pip install -U efficientnet

import efficientnet.keras as efn

     |████████████████████████████████| 51kB 4.0MB/s 


In [ ]:
# Step 1
target_size = (224, 224)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=target_size,  # All images will be resized to 150x150
        batch_size=batch_size,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# Step 2
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')


16809984/16804768 [==============================] - 0s 0us/step


In [ ]:
# Step 3
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Step 4
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(inputs = base_model.input, outputs = predictions)

model_final.compile(RMSprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
# Step 5
eff_history = model_final.fit_generator(train_generator, validation_data = test_generator,
                                        steps_per_epoch = 100, epochs = 10)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
100/100 [==============================] - 38s 318ms/step - loss: 0.5137 - accuracy: 0.8591 - val_loss: 0.1120 - val_accuracy: 0.9730
Epoch 2/10
100/100 [==============================] - 31s 305ms/step - loss: 0.2503 - accuracy: 0.9269 - val_loss: 0.1325 - val_accuracy: 0.9740
Epoch 3/10
100/100 [==============================] - 30s 304ms/step - loss: 0.2221 - accuracy: 0.9497 - val_loss: 0.1579 - val_accuracy: 0.9720
Epoch 4/10
100/100 [==============================] - 31s 305ms/step - loss: 0.1635 - accuracy: 0.9499 - val_loss: 0.1649 - val_accuracy: 0.9790
Epoch 5/10
100/100 [==============================] - 31s 308ms/step - loss: 0.2008 - accuracy: 0.9627 - val_loss: 0.2073 - val_accuracy: 0.9740
Epoch 6/10
100/100 [==============================] - 31s 306ms/step - loss: 0.2361 - accuracy: 0.9413 - val_loss: 0.2639 - val_accuracy: 0.9720
Epoch 7/10
100/100 [==============================] - 31s 305ms/step - loss: 0.2284 - accuracy: 0.9596 - val_loss: 0.3022 - val_ac